In [1]:
import os
import findspark
findspark.init(os.environ['SPARK_HOME'])

In [2]:
import time
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [3]:
def nprint (dstream, text, num=10):
    
    def takeAndPrint (time, rdd):
        taken = rdd.take (num + 1)
        print ("----------------------------------------------")
        print (f'({time}) {text}')
        print ("----------------------------------------------")

        for record in taken[:num]:
            print (record)
        if len(taken) > num:
            print ('...')
        print ('')
        
    dstream.foreachRDD (takeAndPrint)

In [4]:
sc = SparkContext(appName="PythonStreamingQueueStream").getOrCreate()
ssc = StreamingContext(sc, 1)
ssc.checkpoint('/tmp')

rddQueue = [
    ['hola1 como1','hola como2' ],
    ['hola1 como1','hola como2' ],
    ['Tercera Ventana']]

lines = ssc.queueStream(rddQueue)
words = lines.flatMap(lambda line: line.split(" "))
weights = words.map(lambda word: (word, 1))
counts = weights.reduceByKey(lambda x, y: x + y)
def total_general(vals, ctx):
    if ctx is None:
        ctx = 0
    return (sum(vals,ctx))
total = counts.updateStateByKey(total_general)
nprint(total, "Total")
# Lanza el sistema de procesado streaming

ssc.start()

time.sleep(2)   #Depende de intervalo y tam. rddQueue
ssc.stop(stopSparkContext=True, stopGraceFully=True)

Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-62bd741f-4e63-4b1f-890b-ccf43a6f8c6a;1.0
	confs: [default]


:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 361ms :: artifacts dl 10ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2

----------------------------------------------
(2021-12-09 12:21:20) Total
----------------------------------------------
('como1', 1)
('hola1', 1)
('hola', 1)
('como2', 1)

----------------------------------------------
(2021-12-09 12:21:21) Total
----------------------------------------------
('como1', 2)
('hola1', 2)
('hola', 2)
('como2', 2)

----------------------------------------------
(2021-12-09 12:21:22) Total
----------------------------------------------
('como1', 2)
('Ventana', 1)
('hola1', 2)
('hola', 2)
('como2', 2)
('Tercera', 1)

----------------------------------------------
(2021-12-09 12:21:23) Total
----------------------------------------------
('como1', 2)
('Ventana', 1)
('hola1', 2)
('hola', 2)
('como2', 2)
('Tercera', 1)



In [5]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

### References
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#basic-sources
2. https://spark.apache.org/docs/latest/streaming-programming-guide.html#discretized-streams-dstreams
3. https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.StreamingContext